## Finetune Falcon-7b (sharded version) on a Google colab notebook

Project C - Team 4

## Setup

The used libraries are `accelerate`, `peft`, `transformers`, `datasets` and TRL to leverage the recent `SFTTrainer`. We will use `bitsandbytes` to quantize the base model into 4bit (QLoRA approach). We will also install `einops` as it is a requirement to load Falcon models.

In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 14

## Dataset load

We make use of PubMedQA, which has more than 200000 instances (Artificially created)

The dataset can be found [here](https://huggingface.co/datasets/pubmed_qa)

In [ ]:
from datasets import load_dataset

dataset_name = "pubmed_qa"
dataset = load_dataset(dataset_name, "pqa_artificial")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/211269 [00:00<?, ? examples/s]

## Loading the model

In this section we will load the [Falcon 7B model](https://huggingface.co/tiiuae/falcon-7b), quantize it in 4bit and attach LoRA adapters on it. Let's get started!

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "ybelkada/falcon-7b-sharded-bf16"

#Quantizer initialize
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

#Get model from HuggingFace's transformers library
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
#Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Below we will load the configuration file in order to create the LoRA model. According to QLoRA paper, it is important to consider all linear layers in the transformer block for maximum performance. Therefore we will add `dense`, `dense_h_to_4_h` and `dense_4h_to_h` layers in the target modules in addition to the mixed query key value layer.

In [ ]:
from peft import LoraConfig

#Setup numerical value for LoRA
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

#Use LoRA config for fine-tuning this model
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

## Loading the trainer

Here we will use the [`SFTTrainer` from TRL library](https://huggingface.co/docs/trl/main/en/sft_trainer) that gives a wrapper around transformers `Trainer` to easily fine-tune models on instruction based datasets using PEFT adapters. Let's first load the training arguments below.

In [ ]:
from transformers import TrainingArguments

#Arguments needed for training process
output_dir = "falcon-7b-sharded"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
#Paged adamw 32 bits optimization algorithm is used in QLoRA
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [ ]:
#Use 30000 random data from dataset, 30% for test, 70% for training
dataset_reformat = dataset['train'].train_test_split(test_size=9000, train_size=21000)
print(dataset_reformat)

DatasetDict({
    train: Dataset({
        features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
        num_rows: 21000
    })
    test: Dataset({
        features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
        num_rows: 9000
    })
})


In [ ]:
#Format the input
def combine_context(row):
  row['context'] = ''.join(row['context']['contexts'])
  return row

def get_text(row):
  row['text'] = "### Context: " + row['context'] +  "### Question: " + row['question'] + "### Answer: " + row['long_answer']
  return row

dataset_train = dataset_reformat['train']
dataset_train.add_column(name="text", column=dataset_train['long_answer'])
dataset_train = dataset_train.map(combine_context)
for i in range(10):
  print(dataset_train['context'][i])
dataset_train = dataset_train.map(get_text,  remove_columns=dataset_reformat["train"].column_names)

Flow (shear stress)-induced dilation (FD) is attenuated in hypertension. Flow triggers the release by endothelial cells of dilators, such as NO or cyclo-oxygenase (COX) derivatives and constrictor factors such as endothelin-1 (ET-1) which might be involved in several cardiovascular diseases. We hypothesized that ET-1 might play a functional role in FD and participate in the endothelial dysfunction in hypertension.We investigated the effect of chronic treatment with the ETA receptor blocker LU135252 (50 mg/kg/day) for 2 weeks on the dilator response to flow in normotensive (Wistar-Kyoto; WKY) or hypertensive (SHR, n = 7 or 8 per group) rats.Systolic arterial pressure was not significantly affected by chronic ETA receptor blockade in both strains. In mesenteric resistance arteries (diameter: approximately 100 microns), isolated in vitro, FD was lower and myogenic tone higher in SHR than in WKY rats. Chronic ETA receptor blockade increased FD by 73% (7.5 +/- 1.5 to 13.0 +/- 2.7 microns di

In [ ]:
#Get text example
#print(dataset_train['text'][0])
tokens = tokenizer.tokenize("### Human: Can you write a short introduction about the relevance of the term \"monopsony\" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: \"Monopsony\" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, leading to a situation where they are dependent on the employer for their livelihood. This dependence can result in further suppression of wages and a decline in working conditions.\n\nOverall, the concept of monopsony is essential to understanding the dynamics of labor markets and the impact of market power on workers. Further research is needed to understand the extent and impact of monopsonies on the economy and to develop policies to address this issue.\n\nReferences:\nBivens, J., & Mishel, L. (2013). The Pay of Corporate Executives and Financial Professionals as Evidence of Rents in Top 1 Percent Incomes. Journal of Economic Perspectives, 27(3), 57-78.### Human: Now explain it to a dog")
print(tokenizer.convert_tokens_to_ids(tokens))

[19468, 6823, 37, 1853, 299, 3131, 241, 1866, 9705, 544, 248, 23335, 275, 248, 1650, 204, 13, 3032, 387, 1799, 100, 13, 272, 17750, 42, 4012, 745, 6432, 3236, 271, 2879, 988, 387, 1799, 424, 272, 248, 17170, 1208, 273, 27421, 5228, 1959, 25, 19468, 12453, 37, 204, 13, 8350, 387, 1799, 100, 13, 11869, 271, 241, 1208, 4352, 881, 629, 304, 736, 532, 12639, 312, 241, 2057, 822, 379, 1506, 25, 529, 17750, 23, 414, 1650, 304, 4235, 5228, 272, 248, 5477, 1208, 23, 881, 241, 988, 387, 1799, 100, 10679, 504, 2589, 1484, 648, 248, 17324, 273, 1660, 3192, 275, 525, 3900, 25, 390, 5312, 275, 241, 988, 387, 1799, 100, 418, 1226, 272, 3089, 17324, 273, 6511, 8093, 4269, 312, 4862, 23, 345, 248, 10679, 504, 1278, 19785, 271, 2638, 17324, 379, 1586, 1286, 1660, 3192, 25, 193, 193, 15770, 1959, 504, 6935, 2879, 988, 387, 1799, 424, 272, 9462, 963, 345, 5543, 273, 2794, 1655, 23, 881, 241, 1218, 1902, 2404, 1873, 241, 2589, 8091, 275, 248, 1208, 204, 19, 45, 419, 594, 204, 17, 22399, 2776, 23, 204, 626,

Then finally pass everthing to the trainer

In [ ]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_train,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/21000 [00:00<?, ? examples/s]

We will also pre-process the model by upcasting the layer norms in float 32 for more stable training

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

## Train the model

Now let's train the model! Simply call `trainer.train()`

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


IndexError: ignored

During training, the model should converge nicely as follows:

![image](https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/loss-falcon-7b.png)

The `SFTTrainer` also takes care of properly saving only the adapters during training instead of saving the entire model.

In [ ]:
!pip install huggingface_hub

In [ ]:
#Login to huggingface
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
#Push model to hub
trainer.push_to_hub("falcon-7b-sharded")